# Mapping digit-representations in the human primary somatosensory cortex and their layer-dependent functional connectivity at 7T.
## Preprocessing

In this notebook, we are describing and, where possible, performing the preprocessing of this publication.

In [16]:
# basic
import glob
import itertools
import pandas as pd
import numpy as np
import os

# plotting
from nilearn import plotting
from scipy import ndimage
import matplotlib.cm as cm
import matplotlib.image as mpimg
import seaborn as sns
import matplotlib.pyplot as plt

# neuroimaging
from nilearn.connectome import ConnectivityMeasure
import nibabel as nb
from nibabel import load, save, Nifti1Image
import nipype.interfaces.fsl as fsl
import nilearn
import ants

# misc
from scipy.ndimage import morphology
from scipy import stats
from scipy import interpolate


%matplotlib inline

210806-20:37:45,563 nipype.utils INFO:
	 Running nipype version 1.4.2 (latest: 1.5.1)


## locating the data

Because the data location will be different for every machine, we will first define a root folder that we will use from then on.

In [ ]:
root = '/media/sebastian/Data/S1ANFUNCO/raw_data'

In this folder, we will find the find the DICOMs, the nifti data in BIDS format in the "S1ANFUNCO_BIDS" folder and the derivatives folder, which will contain our processed data and analyses.


Next, we can make a list of participant-identifiers:

In [ ]:
subFolders = sorted(glob.glob(root + '/S1ANFUNCO_BIDS/sub*'))
subs = [i[-6:] for i in subFolders]

In [ ]:
subs

In [ ]:
sessFolders = sorted(glob.glob(root + '/S1ANFUNCO_BIDS/sub*/*'))
sessFolders

What comes to mind immediately are the participant and session numbers. When scanning the first 5 subjects, we tested different imaging-parameters and stimulation protocols. Therefore, the subjects are either not included or were reinvited for at a later time. Hence, the higher session numbers in sub-02 and sub-05.

In the derivatives folder, we will have to make a new folder for each subject.

In [ ]:
%%bash -s "{" ".join(subs)}" "$root"

for sub in $1
do
mkdir $2/derivatives/$sub
mkdir $2/derivatives/$sub/anat
mkdir $2/derivatives/$sub/func
mkdir $2/derivatives/$sub/func/rest
mkdir $2/derivatives/$sub/func/stim
done

## Motion correction

To account for non-linear distortions due to low bandwidth, a manually drawn motion mask covering the postcentral gyrus was used to optimize registration on our target region. Mask selection was guided by the raw EPI data and they can be inspected in the derivatives folder.

In [ ]:
motionMasks = sorted(glob.glob('/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/sub-*/sub-*_moma.nii'))
restRuns = sorted(glob.glob('/media/sebastian/Data/S1ANFUNCO/raw_data/S1ANFUNCO_BIDS/sub-*/ses-00*/func/sub-*_ses-00*_task-restBA3b_run-001_cbv.nii.gz'))

for restRun, motionMask in zip(restRuns,motionMasks):

    underlay = nb.load(restRun).get_fdata()[:,:,:,0]
    
    display = plotting.plot_roi(motionMask, underlay, display_mode='z')

### Resting state

For each subject, we want to register all runs to the resting state data. Therefore, we will first do the motion correction for those.

To loop properly in bash, I need to create various lists in python first

In [ ]:
subFolders = sorted(glob.glob(root + '/S1ANFUNCO_BIDS/sub*'))
subs = [i[-6:] for i in subFolders]
restRuns = sorted(glob.glob('/media/sebastian/Data/S1ANFUNCO/raw_data/S1ANFUNCO_BIDS/sub-*/ses-00*/func/sub-*_ses-00*_task-restBA3b_run-001_cbv.nii.gz'))
motionMasks = sorted(glob.glob('/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/sub-*/sub-*_moma.nii'))
outFolders =  sorted(glob.glob('/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/sub-*/func/rest'))

In [ ]:
%%bash -s "{" ".join(subs)}" "{" ".join(restRuns)}" "{" ".join(motionMasks)}" "{" ".join(outFolders)}"


List1=$1
List2=$2
List3=$3
List4=$4

subsList=($List1)
restRunsList=($List2)
momaList=($List3)
outFolders=($List4)

len=${#subsList[@]}
 
for (( j=0; j<$len; j++ ))
do

base=${subsList[$j]}_rest3b_run-001
outFolder=${outFolders[$j]}
echo Current subject: ${subsList[$j]}
echo The resting-state data is: ${restRunsList[$j]}
echo The motion mask is: ${momaList[$j]}
echo The output will be saved here: ${outFolder}
echo The basename of the output is: ${base}
echo #
echo #
echo #

cd ${outFolder}


###########################################
###### parameters that are not used  ######
###########################################

ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS=8
export ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS
tr=1
basevol=1000 # ANTs indexing

mkdir -p motionParameters/run-001/nn_motion
mkdir -p motionParameters/run-001/n_motion


###########################################
####### preparig odd and even images ######
###########################################

#for classical Magentom VASO sequences
3dcalc -a ${restRunsList[$j]}'[0..$(2)]' -expr 'a' -prefix ${base}_notnulled.nii -overwrite
3dcalc -a ${restRunsList[$j]}'[1..$(2)]' -expr 'a' -prefix ${base}_nulled.nii -overwrite

# for Terra 7T VASO sequence
#3dcopy $1'[0..$(2)]' nulled.nii
#3dcopy $1'[1..$(2)]' notnulled.nii


# 3dAutomask -prefix moma.nii -peels 3 -dilate 2  notnulled.nii

###########################################
####### Do MOCO on notnulled  #############
###########################################

n_vols=`PrintHeader ${base}_notnulled.nii | grep Dimens | cut -d ',' -f 4 | cut -d ']' -f 1`

echo "seperating $n_vols time steps to save RAM"
ImageMath 4 vol_.nii TimeSeriesDisassemble ${base}_notnulled.nii # vol_1000.nii, vol_1001.nii ...
cp vol_1003.nii vol_1000.nii # removing steady state effects
cp vol_1004.nii vol_1001.nii
cp vol_1005.nii vol_1002.nii
3dMean -prefix ${base}_nn_reference.nii vol_1004.nii vol_1005.nii # there is no overwrite here, in case of multiple series
nthvol=$(($basevol + $n_vols - 1)) # Zero indexing
echo "doing the alignemt"
for i in $(eval echo "{$basevol..$nthvol}");
do
antsRegistration \
--dimensionality 3 \
--float 1 \
--collapse-output-transforms 1 \
--output [ vol_${i}_,vol_${i}_Warped.nii.gz,vol_${i}_InverseWarped.nii.gz ] \
--interpolation BSpline[2] \
--use-histogram-matching 1 \
--winsorize-image-intensities [ 0.005,0.995 ] \
-x ${momaList[$i]} \
--initial-moving-transform [ ${base}_nn_reference.nii,vol_${i}.nii,1 ] \
--transform Rigid[ 0.1 ] \
--metric MI[ ${base}_nn_reference.nii,vol_${i}.nii,1,32,Regular,0.25 ] \
--convergence [ 250x100,1e-6,10 ] \
--shrink-factors 2x1 \
--smoothing-sigmas 1x0vox
#--transform SyN[ 0.1,3,0 ] \
#--metric CC[ nn_reference.nii,vol_${i}.nii,1,4 ] \
#--convergence [ 50x0,1e-6,10 ] \
#--shrink-factors 2x1 \
#--smoothing-sigmas 1x0vox
done

echo "reassembling the time points"
ImageMath 4 ${base}_moco_notnulled.nii TimeSeriesAssemble $tr 0 vol_*_Warped.nii.gz
mv vol_*.nii motionParameters/run-001/nn_motion
mv vol_*_0GenericAffine.mat motionParameters/run-001/nn_motion
mv vol_*_InverseWarped.nii.gz motionParameters/run-001/nn_motion
mv vol_*_Warped.nii.gz motionParameters/run-001/nn_motion


###########################################
####### Do MOCO on nulled  ################
###########################################

n_vols=`PrintHeader ${base}_nulled.nii | grep Dimens | cut -d ',' -f 4 | cut -d ']' -f 1`

echo "seperating $n_vols time steps to save RAM"
ImageMath 4 vol_.nii TimeSeriesDisassemble ${base}_nulled.nii # vol_1000.nii, vol_1001.nii ...
cp vol_1003.nii vol_1000.nii # removing steady state effects
cp vol_1004.nii vol_1001.nii
cp vol_1005.nii vol_1002.nii
3dMean -prefix ${base}_n_reference.nii vol_1004.nii vol_1005.nii # there is no overwrite here, in case of multiple series
nthvol=$(($basevol + $n_vols - 1)) # Zero indexing
echo "doing the alignemt"
for i in $(eval echo "{$basevol..$nthvol}");
do
antsRegistration \
--dimensionality 3 \
--float 1 \
--collapse-output-transforms 1 \
--output [ vol_${i}_,vol_${i}_Warped.nii.gz,vol_${i}_InverseWarped.nii.gz ] \
--interpolation BSpline[2] \
--use-histogram-matching 1 \
--winsorize-image-intensities [ 0.005,0.995 ] \
-x ${momaList[$i]} \
--initial-moving-transform [ ${base}_n_reference.nii,vol_${i}.nii,1 ] \
--transform Rigid[ 0.1 ] \
--metric MI[ ${base}_n_reference.nii,vol_${i}.nii,1,32,Regular,0.25 ] \
--convergence [ 250x100,1e-6,10 ] \
--shrink-factors 2x1 \
--smoothing-sigmas 1x0vox
#--transform SyN[ 0.1,3,0 ] \
#--metric CC[ n_reference.nii,vol_${i}.nii,1,4 ] \
#--convergence [ 50x0,1e-6,10 ] \
#--shrink-factors 2x1 \
#--smoothing-sigmas 1x0voxdone
done
echo "reassembling the time points"
ImageMath 4 ${base}_moco_nulled.nii TimeSeriesAssemble $tr 0 vol_*_Warped.nii.gz
mv vol_*.nii motionParameters/run-001/n_motion
mv vol_*_0GenericAffine.mat motionParameters/run-001/n_motion
mv vol_*_Warped.nii.gz motionParameters/run-001/n_motion
mv vol_*_InverseWarped.nii.gz motionParameters/run-001/n_motion


##############################
#### fsl_motion_outliers  ####
##############################

fsl_motion_outliers -i ${base}_nulled.nii -o motionParameters/run-001/${base}_nulled_confounds.txt
fsl_motion_outliers -i ${base}_notnulled.nii -o motionParameters/run-001/${base}_notnulled_confounds.txt


##################################################
#### Temporal upsampling so VASO matches BOLD ####
##################################################

echo "temporal upsampling and shifting happens now"
3dUpsample -overwrite  -datum short -prefix ${base}_Nulled_intemp.nii -n 2 -input ${base}_moco_nulled.nii
3dUpsample -overwrite  -datum short -prefix ${base}_BOLD_intemp.nii   -n 2 -input ${base}_moco_notnulled.nii

##for Magentom clasical VASO sequence
NumVol=`3dinfo -nv ${base}_Nulled_intemp.nii`
3dTcat -overwrite -prefix ${base}_Nulled_intemp.nii ${base}_Nulled_intemp.nii'[0]' ${base}_Nulled_intemp.nii'[0..'`expr $NumVol - 2`']'

echo "I am correcting for the proper TR in the header"
3drefit -TR 1.9295 ${base}_BOLD_intemp.nii
3drefit -TR 1.9295 ${base}_Nulled_intemp.nii

echo "BOLD correction happens now"
LN_BOCO -Nulled ${base}_Nulled_intemp.nii -BOLD ${base}_BOLD_intemp.nii -output ${base}

echo "calculating T1 in EPI space"
3dTcat -prefix combined.nii  ${base}_moco_nulled.nii ${base}_moco_notnulled.nii -overwrite
3dTstat -cvarinv -overwrite  -prefix ${base}_T1w.nii combined.nii
rm combined.nii

echo "curtosis and skew"
LN_SKEW -input ${base}_BOLD_intemp.nii
LN_SKEW -input ${base}_VASO_LN.nii

# The FSL GLM does not seem to like images where values are between 0 and 1. There fore, we are multipolying everything by 100.
fslmaths ${base}_VASO_LN.nii -mul 100 ${base}_VASO_LN_short.nii -odt short


done

### Stimulation 

After the motion correction is completed for the resting state data, we can use the reference images for the stimulation data. However, we have to take care here, because some participants underwent multiple runs of stimulation.

In [ ]:
stimRuns = sorted(glob.glob('/media/sebastian/Data/S1ANFUNCO/raw_data/S1ANFUNCO_BIDS/sub-*/ses-00*/func/sub-*_ses-00*_task-stim*_cbv.nii.gz'))
runNumberList = [i[-18:-11] for i in stimRuns]
subList = [i[-43:-37] for i in stimRuns]

In [ ]:
%%bash -s "{" ".join(subList)}" "{" ".join(stimRuns)}" "{" ".join(runNumberList)}" 


List1=$1
List2=$2
List3=$3

subsList=($List1)
stimRunsList=($List2)
runNumberList=($List3)

len=${#subsList[@]}
 
for (( j=0; j<$len; j++ ))
do

base=${subsList[$j]}_stim_${runNumberList[$j]}
outFolder=/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/${subsList[$j]}/func/stim
echo Current subject: ${subsList[$j]}
echo The current stimulation run is: ${stimRunsList[$j]}
echo The motion mask is: /media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/${subsList[$j]}/${subsList[$j]}_moma.nii
echo The output will be saved here: ${outFolder}
echo The basename of the output is: ${base}
echo #
echo #
echo #


cd ${outFolder}


###########################################
###### parameters that are not used  ######
###########################################

ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS=8
export ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS
tr=1
basevol=1000 # ANTs indexing

mkdir -p motionParameters/${runNumberList[$j]}/nn_motion
mkdir -p motionParameters/${runNumberList[$j]}/n_motion


###########################################
####### preparig odd and even images ######
###########################################

#for classical Magentom VASO sequences
3dcalc -a ${stimRunsList[$j]}'[0..$(2)]' -expr 'a' -prefix ${base}_notnulled.nii -overwrite
3dcalc -a ${stimRunsList[$j]}'[1..$(2)]' -expr 'a' -prefix ${base}_nulled.nii -overwrite


###########################################
####### Do MOCO on notnulled  #############
###########################################

n_vols=`PrintHeader ${base}_notnulled.nii | grep Dimens | cut -d ',' -f 4 | cut -d ']' -f 1`

echo "seperating $n_vols time steps to save RAM"
ImageMath 4 vol_.nii TimeSeriesDisassemble ${base}_notnulled.nii # vol_1000.nii, vol_1001.nii ...
cp vol_1003.nii vol_1000.nii # removing steady state effects
cp vol_1004.nii vol_1001.nii
cp vol_1005.nii vol_1002.nii
nthvol=$(($basevol + $n_vols - 1)) # Zero indexing
echo "doing the alignemt"
for i in $(eval echo "{$basevol..$nthvol}");
do
antsRegistration \
--dimensionality 3 \
--float 1 \
--collapse-output-transforms 1 \
--output [ vol_${i}_,vol_${i}_Warped.nii.gz,vol_${i}_InverseWarped.nii.gz ] \
--interpolation BSpline[2] \
--use-histogram-matching 1 \
--winsorize-image-intensities [ 0.005,0.995 ] \
-x ${momaList[$i]} \
--initial-moving-transform [ ../rest/${subsList[$j]}_rest3b_run-001_nn_reference.nii,vol_${i}.nii,1 ] \
--transform Rigid[ 0.1 ] \
--metric MI[ ../rest/${subsList[$j]}_rest3b_run-001_nn_reference.nii,vol_${i}.nii,1,32,Regular,0.25 ] \
--convergence [ 250x100,1e-6,10 ] \
--shrink-factors 2x1 \
--smoothing-sigmas 1x0vox
#--transform SyN[ 0.1,3,0 ] \
#--metric CC[ nn_reference.nii,vol_${i}.nii,1,4 ] \
#--convergence [ 50x0,1e-6,10 ] \
#--shrink-factors 2x1 \
#--smoothing-sigmas 1x0vox
done

echo "reassembling the time points"
ImageMath 4 ${base}_moco_notnulled.nii TimeSeriesAssemble $tr 0 vol_*_Warped.nii.gz
mv vol_*.nii motionParameters/${runNumberList[$j]}/nn_motion
mv vol_*_0GenericAffine.mat motionParameters/${runNumberList[$j]}/nn_motion
mv vol_*_1Warp.nii.gz motionParameters/${runNumberList[$j]}/nn_motion
mv vol_*_1InverseWarp.nii.gz motionParameters/${runNumberList[$j]}/nn_motion
mv vol_*_InverseWarped.nii.gz motionParameters/${runNumberList[$j]}/nn_motion
mv vol_*_Warped.nii.gz motionParameters/${runNumberList[$j]}/nn_motion


###########################################
####### Do MOCO on nulled  ################
###########################################

n_vols=`PrintHeader ${base}_nulled.nii | grep Dimens | cut -d ',' -f 4 | cut -d ']' -f 1`

echo "seperating $n_vols time steps to save RAM"
ImageMath 4 vol_.nii TimeSeriesDisassemble ${base}_nulled.nii # vol_1000.nii, vol_1001.nii ...
cp vol_1003.nii vol_1000.nii # removing steady state effects
cp vol_1004.nii vol_1001.nii
cp vol_1005.nii vol_1002.nii
nthvol=$(($basevol + $n_vols - 1)) # Zero indexing
echo "doing the alignemt"
for i in $(eval echo "{$basevol..$nthvol}");
do
antsRegistration \
--dimensionality 3 \
--float 1 \
--collapse-output-transforms 1 \
--output [ vol_${i}_,vol_${i}_Warped.nii.gz,vol_${i}_InverseWarped.nii.gz ] \
--interpolation BSpline[2] \
--use-histogram-matching 1 \
--winsorize-image-intensities [ 0.005,0.995 ] \
-x ${momaList[$i]} \
--initial-moving-transform [ ../rest/${subsList[$j]}_rest3b_run-001_n_reference.nii,vol_${i}.nii,1 ] \
--transform Rigid[ 0.1 ] \
--metric MI[ ../rest/${subsList[$j]}_rest3b_run-001_n_reference.nii,vol_${i}.nii,1,32,Regular,0.25 ] \
--convergence [ 250x100,1e-6,10 ] \
--shrink-factors 2x1 \
--smoothing-sigmas 1x0vox
#--transform SyN[ 0.1,3,0 ] \
#--metric CC[ n_reference.nii,vol_${i}.nii,1,4 ] \
#--convergence [ 50x0,1e-6,10 ] \
#--shrink-factors 2x1 \
#--smoothing-sigmas 1x0voxdone
done
echo "reassembling the time points"
ImageMath 4 ${base}_moco_nulled.nii TimeSeriesAssemble $tr 0 vol_*_Warped.nii.gz
mv vol_*.nii motionParameters/${runNumberList[$j]}/n_motion
mv vol_*_0GenericAffine.mat motionParameters/${runNumberList[$j]}/n_motion
mv vol_*_Warped.nii.gz motionParameters/${runNumberList[$j]}/n_motion
mv vol_*__InverseWarped.nii.gz motionParameters/${runNumberList[$j]}/n_motion
mv vol_*_InverseWarped.nii.gz motionParameters/${runNumberList[$j]}/n_motion
mv vol_*_Warped.nii.gz motionParameters/${runNumberList[$j]}/n_motion


##############################
#### fsl_motion_outliers  ####
##############################

fsl_motion_outliers -i ${base}_nulled.nii -o motionParameters/${runNumberList[$j]}/${base}_nulled_confounds.txt
fsl_motion_outliers -i ${base}_notnulled.nii -o motionParameters/${runNumberList[$j]}/${base}_notnulled_confounds.txt



##################################################
#### Temporal upsampling so VASO matches BOLD ####
##################################################

echo "temporal upsampling and shifting happens now"
3dUpsample -overwrite  -datum short -prefix ${base}_Nulled_intemp.nii -n 2 -input ${base}_moco_nulled.nii
3dUpsample -overwrite  -datum short -prefix ${base}_BOLD_intemp.nii   -n 2 -input ${base}_moco_notnulled.nii

##for Magentom clasical VASO sequence
NumVol=`3dinfo -nv ${base}_Nulled_intemp.nii`
3dTcat -overwrite -prefix ${base}_Nulled_intemp.nii ${base}_Nulled_intemp.nii'[0]' ${base}_Nulled_intemp.nii'[0..'`expr $NumVol - 2`']'

echo "I am correcting for the proper TR in the header"
3drefit -TR 1.9295 ${base}_BOLD_intemp.nii
3drefit -TR 1.9295 ${base}_Nulled_intemp.nii

echo "BOLD correction happens now"
LN_BOCO -Nulled ${base}_Nulled_intemp.nii -BOLD ${base}_BOLD_intemp.nii -output ${base}

echo "calculating T1 in EPI space"
3dTcat -prefix combined.nii  ${base}_moco_nulled.nii ${base}_moco_notnulled.nii -overwrite
3dTstat -cvarinv -overwrite  -prefix ${base}_T1w.nii combined.nii
rm combined.nii

echo "curtosis and skew"
LN_SKEW -input ${base}_BOLD_intemp.nii
LN_SKEW -input ${base}_VASO_LN.nii

# The FSL GLM does not seem to like images where values are between 0 and 1. There fore, we are multipolying everything by 100.
fslmaths ${base}_VASO_LN.nii -mul 100 ${base}_VASO_LN_short.nii -odt short




done

## Motion-Parameters

Here, we want to get the motion-parameters. Because ANTs does not save real world parameters, we will transfer the affine matrices into FSL format.

In [ ]:
subFolders = sorted(glob.glob(root + '/S1ANFUNCO_BIDS/sub*'))
subs = [i[-6:] for i in subFolders]
restRuns = sorted(glob.glob('/media/sebastian/Data/S1ANFUNCO/raw_data/S1ANFUNCO_BIDS/sub-*/ses-00*/func/sub-*_ses-00*_task-restBA3b_run-001_cbv.nii.gz'))
motionMasks = sorted(glob.glob('/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/sub-*/sub-*_moma.nii'))
outFolders =  sorted(glob.glob('/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/sub-*/func/rest'))

In [ ]:
%%bash -s "{" ".join(subs)}" "{" ".join(restRuns)}" "{" ".join(outFolders)}"


List1=$1
List2=$2
List3=$3


subsList=($List1)
restRunsList=($List2)
outFolders=($List3)

len=${#subsList[@]}


imageTypes=("n" "nn")


for (( j=0; j<$len; j++ ))
do

# base=${subsList[$j]}_rest3b
base=${subsList[$j]}_rest3b_run-001
outFolder=${outFolders[$j]}
echo Current subject: ${subsList[$j]}
echo The output will be saved here: ${outFolder}
echo The basename of the output is: ${base}
echo #
echo #
echo #

cd ${outFolder}

n_vols=`PrintHeader ${base}_notnulled.nii | grep Dimens | cut -d ',' -f 4 | cut -d ']' -f 1`
num=$((n_vols + 1000))

echo $num

for imageType in "${imageTypes[@]}"
do


for (( k=1000; k<$num; k++ ))
do
ConvertTransformFile \
3 \
''${outFolder}'/motionParameters/run-001/'$imageType'_motion/vol_'$k'_0GenericAffine.mat' \
''${outFolder}'/motionParameters/run-001/'$imageType'_motion/vol_'$k'_0GenericAffine_af.mat' \
--convertToAffineType


c3d_affine_tool \
-ref ''${subsList[$j]}'_rest3b_run-001_'$imageType'_reference.nii' \
-src ''${outFolder}'/motionParameters/run-001/'$imageType'_motion/vol_'$k'.nii' \
-itk ''${outFolder}'/motionParameters/run-001/'$imageType'_motion/vol_'$k'_0GenericAffine_af.mat' -ras2fsl \
-o ''${outFolder}'/motionParameters/run-001/'$imageType'_motion/vol_'$k'_0GenericAffine_FSL.mat' \
-info-full
done

done

done

Then we can read the matrices and extract the motion traces.

Because the output of the cell made the file-size explode, I will supress it here.

In [ ]:
%%capture

lstsubMot  = []
lstsubMot_Nme  = []
lstTR_sub = []
sub_list =[]
modalityList = []

modalities = ["n", 'nn']
modalitiesDict = {'n': 'VASO', 'nn': 'BOLD'}


for sub in subs:
    
    for modality in modalities:
        
        print(sub)
        affinemats = sorted(glob.glob(root + f'/derivatives/{sub}/func/rest/motionParameters/run-001/{modality}_motion/vol_*_0GenericAffine_FSL.mat'))
        print(len(affinemats))

        volids=[]
        for n in range(len(affinemats)):
            volids.append(n+1000)

        for volid in volids:

            # Current mats
            currMats = root + f'/derivatives/{sub}/func/rest/motionParameters/run-001/{modality}_motion/vol_{volid}_0GenericAffine_FSL.mat'

            tmp = fsl.AvScale(all_param=True,mat_file=currMats)

            tmpReadout = tmp.run()

            # Get the rotations (in rads) and translations (in mm) per volume

            aryTmpMot = list(itertools.chain.from_iterable(
                                        [tmpReadout.outputs.translations,
                                         tmpReadout.outputs.rot_angles]))




            # Save the roation and translations
            lstsubMot.append(aryTmpMot)
            lstTR_sub.append([int(volid)+1-1000 for i in range(6)])               
            lstsubMot_Nme.append([f'TX {modalitiesDict[modality]}',f'TY {modalitiesDict[modality]}',f'TZ {modalitiesDict[modality]}',f'RX {modalitiesDict[modality]}',f'RY {modalitiesDict[modality]}',f'RZ {modalitiesDict[modality]}'])
            sub_list.append([str(sub) for i in range(6)])
            modalityList.append([str(modalitiesDict[modality]) for i in range(6)])

Convert the lists into arrays and them make a nice dataframe.

In [ ]:
aryCurr = np.array(lstsubMot)
aryCurr_Ses =  aryCurr.reshape((aryCurr.size,-1))
aryCurr_TR = np.array(lstTR_sub)
aryCurr_TR_Ses = aryCurr_TR.reshape((aryCurr_TR.size,-1))
aryCurr_Nme = np.array(lstsubMot_Nme)
aryCurr_Nme_Ses = aryCurr_Nme.reshape((aryCurr_Nme.size,-1))
aryIdx = np.arange(1,len(aryCurr_Nme_Ses)+1)

aryCurr_mod = np.array(modalityList)
aryCurr_mod = aryCurr_mod.reshape((aryCurr_mod.size,-1))

aryCurr_subname = np.array(sub_list)
aryCurr_subs =  aryCurr_subname.reshape((aryCurr_subname.size,-1))
     
data_dict = {
    'subject': aryCurr_subs[:,0],
    'Time/TR': aryCurr_TR_Ses[:,0],
    'Motion_Name': aryCurr_Nme_Ses[:,0],
    'Motion': aryCurr_Ses[:,0],
    'idx':aryIdx,
    'modality': aryCurr_mod[:,0]}
    
pd_ses = pd.DataFrame(data=data_dict)


Which can then be saved.

In [ ]:
pd_ses.to_csv(root + '/derivatives/quality_assessmen/motionParametersRest.csv', index=False)  

This is then repeated for the stimulation data.

In [ ]:
stimRuns = sorted(glob.glob('/media/sebastian/Data/S1ANFUNCO/raw_data/S1ANFUNCO_BIDS/sub-*/ses-00*/func/sub-*_ses-00*_task-stim*_cbv.nii.gz'))
runNumberList = [i[-18:-11] for i in stimRuns]
subList = [i[-43:-37] for i in stimRuns]
outFolders =  sorted(glob.glob('/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/sub-*/func/stim/motionParameters/run-00*'))

In [ ]:
%%bash -s "{" ".join(subList)}" "{" ".join(stimRuns)}" "{" ".join(runNumberList)}" "{" ".join(outFolders)}"


List1=$1
List2=$2
List3=$3
List4=$4

subsList=($List1)
stimRunsList=($List2)
runNumberList=($List3)
outFolders=($List4)

len=${#subsList[@]}



imageTypes=("n" "nn")

len=${#subsList[@]}
 

for (( j=0; j<$len; j++ ))
do

base=${subsList[$j]}_stim_${runNumberList[$j]}
outFolder=${outFolders[$j]}
echo Current subject: ${subsList[$j]}
echo The current stimulation run is: ${stimRunsList[$j]}
echo The output will be saved here: ${outFolder}
echo The basename of the output is: ${base}
echo #
echo #
echo #

cd ${outFolder}

n_vols=`PrintHeader ../../${base}_notnulled.nii | grep Dimens | cut -d ',' -f 4 | cut -d ']' -f 1`
num=$((n_vols + 1000))

echo $num

for imageType in "${imageTypes[@]}"
do


for (( k=1000; k<$num; k++ ))
do
ConvertTransformFile \
3 \
''$imageType'_motion/vol_'$k'_0GenericAffine.mat' \
''$imageType'_motion/vol_'$k'_0GenericAffine_af.mat' \
--convertToAffineType


c3d_affine_tool \
-ref '../../../rest/'${subsList[$j]}'_rest3b_run-001_'$imageType'_reference.nii' \
-src ''$imageType'_motion/vol_'$k'.nii' \
-itk ''$imageType'_motion/vol_'$k'_0GenericAffine_af.mat' -ras2fsl \
-o ''$imageType'_motion/vol_'$k'_0GenericAffine_FSL.mat' \
-info-full
done

done

done

In [17]:
stimRuns = sorted(glob.glob('/media/sebastian/Data/S1ANFUNCO/raw_data/S1ANFUNCO_BIDS/sub-*/ses-00*/func/sub-*_ses-00*_task-stim*_cbv.nii.gz'))
runNumberList = [i[-18:-11] for i in stimRuns]
subList = [i[-43:-37] for i in stimRuns]
outFolders =  sorted(glob.glob('/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/sub-*/func/stim/motionParameters/run-00*'))

In [22]:
%%capture

lstsubMot  = []
lstsubMot_Nme  = []
lstTR_sub = []
sub_list =[]
modalityList = []
runsList = []

modalities = ["n", 'nn']
modalitiesDict = {'n': 'VASO', 'nn': 'BOLD'}


for sub in subList:
    for run in runNumberList:

        for modality in modalities:

            print(sub)
            affinemats = sorted(glob.glob(root + f'/derivatives/{sub}/func/stim/motionParameters/{run}/{modality}_motion/vol_*_0GenericAffine_FSL.mat'))
            print(len(affinemats))

            volids=[]
            for n in range(len(affinemats)):
                volids.append(n+1000)

            for volid in volids:

                # Current mats
                currMats = root + f'/derivatives/{sub}/func/stim/motionParameters/{run}/{modality}_motion/vol_{volid}_0GenericAffine_FSL.mat'

                tmp = fsl.AvScale(all_param=True,mat_file=currMats);

                tmpReadout = tmp.run();

                # Get the rotations (in rads) and translations (in mm) per volume

                aryTmpMot = list(itertools.chain.from_iterable([tmpReadout.outputs.translations,tmpReadout.outputs.rot_angles]));




                # Save the roation and translations
                lstsubMot.append(aryTmpMot)
                lstTR_sub.append([int(volid)+1-1000 for i in range(6)])               
                lstsubMot_Nme.append([f'TX {modalitiesDict[modality]}',f'TY {modalitiesDict[modality]}',f'TZ {modalitiesDict[modality]}',f'RX {modalitiesDict[modality]}',f'RY {modalitiesDict[modality]}',f'RZ {modalitiesDict[modality]}'])
                sub_list.append([str(sub) for i in range(6)])
                modalityList.append([str(modalitiesDict[modality]) for i in range(6)])
                runsList.append([str(run) for i in range(6)])

210806-20:43:36,473 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.472283:Rotation & Translation Matrix:
210806-20:43:36,475 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.472283:0.999986 0.005272 -0.000990 -0.127009 
210806-20:43:36,476 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.472283:-0.005284 0.999908 -0.012524 -0.027541 
210806-20:43:36,477 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.472283:0.000924 0.012529 0.999921 -0.928744 
210806-20:43:36,478 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.472283:0.000000 0.000000 0.000000 1.000000 
210806-20:43:36,479 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.472283:
210806-20:43:36,480 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.472283:Rotation Angles (x,y,z) [rads] = -0.012525 0.000990 0.005272 
210806-20:43:36,480 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.472283:
210806-20:43:36,481 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.472283:Translations (x,y,z) [mm] = -0.127009 -0

210806-20:43:36,961 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.948323:
210806-20:43:36,961 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.948323:Determinant = 1.000000
210806-20:43:36,962 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.948323:Left-Right orientation: preserved
210806-20:43:36,963 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.948323:
210806-20:43:36,963 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.948323:Forward half transform =
210806-20:43:36,964 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.948323:0.999996 0.002622 -0.000338 -0.036137 
210806-20:43:36,964 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.948323:-0.002624 0.999978 -0.006066 -0.008761 
210806-20:43:36,965 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.948323:0.000322 0.006067 0.999981 -0.410187 
210806-20:43:36,965 nipype.interface INFO:
	 stdout 2021-08-06T20:43:36.948323:0.000000 0.000000 0.000000 1.000000 
210806-20:43:36,966 nipype.interface INFO:
	 stdout

210806-20:43:37,430 nipype.interface INFO:
	 stdout 2021-08-06T20:43:37.424171:
210806-20:43:37,637 nipype.interface INFO:
	 stdout 2021-08-06T20:43:37.636912:Rotation & Translation Matrix:
210806-20:43:37,639 nipype.interface INFO:
	 stdout 2021-08-06T20:43:37.636912:0.999985 0.005362 -0.000863 -0.079688 
210806-20:43:37,640 nipype.interface INFO:
	 stdout 2021-08-06T20:43:37.636912:-0.005372 0.999911 -0.012198 -0.012948 
210806-20:43:37,640 nipype.interface INFO:
	 stdout 2021-08-06T20:43:37.636912:0.000797 0.012202 0.999925 -0.841803 
210806-20:43:37,641 nipype.interface INFO:
	 stdout 2021-08-06T20:43:37.636912:0.000000 0.000000 0.000000 1.000000 
210806-20:43:37,642 nipype.interface INFO:
	 stdout 2021-08-06T20:43:37.636912:
210806-20:43:37,642 nipype.interface INFO:
	 stdout 2021-08-06T20:43:37.636912:Rotation Angles (x,y,z) [rads] = -0.012198 0.000863 0.005362 
210806-20:43:37,643 nipype.interface INFO:
	 stdout 2021-08-06T20:43:37.636912:
210806-20:43:37,644 nipype.interface IN

210806-20:43:38,105 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.093470:
210806-20:43:38,105 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.093470:Determinant = 1.000000
210806-20:43:38,106 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.093470:Left-Right orientation: preserved
210806-20:43:38,106 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.093470:
210806-20:43:38,107 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.093470:Forward half transform =
210806-20:43:38,107 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.093470:0.999997 0.002446 0.000120 -0.033067 
210806-20:43:38,108 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.093470:-0.002445 0.999982 -0.005436 -0.011244 
210806-20:43:38,108 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.093470:-0.000133 0.005436 0.999985 -0.280141 
210806-20:43:38,109 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.093470:0.000000 0.000000 0.000000 1.000000 
210806-20:43:38,109 nipype.interface INFO:
	 stdout

210806-20:43:38,559 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.536683:
210806-20:43:38,769 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.769021:Rotation & Translation Matrix:
210806-20:43:38,770 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.769021:0.999990 0.004463 -0.000494 -0.064673 
210806-20:43:38,771 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.769021:-0.004468 0.999942 -0.009767 0.029381 
210806-20:43:38,772 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.769021:0.000451 0.009769 0.999952 -0.718161 
210806-20:43:38,773 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.769021:0.000000 0.000000 0.000000 1.000000 
210806-20:43:38,774 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.769021:
210806-20:43:38,774 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.769021:Rotation Angles (x,y,z) [rads] = -0.009767 0.000494 0.004463 
210806-20:43:38,775 nipype.interface INFO:
	 stdout 2021-08-06T20:43:38.769021:
210806-20:43:38,776 nipype.interface INF

...

210806-20:43:42,255 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.255422:Rotation & Translation Matrix:
210806-20:43:42,257 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.255422:0.999992 0.003951 -0.000177 0.033059 
210806-20:43:42,258 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.255422:-0.003952 0.999971 -0.006531 -0.032035 
210806-20:43:42,259 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.255422:0.000151 0.006532 0.999979 -0.426509 
210806-20:43:42,260 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.255422:0.000000 0.000000 0.000000 1.000000 
210806-20:43:42,261 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.255422:
210806-20:43:42,262 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.255422:Rotation Angles (x,y,z) [rads] = -0.006531 0.000177 0.003951 
210806-20:43:42,263 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.255422:
210806-20:43:42,264 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.255422:Translations (x,y,z) [mm] = 0.033059 -0.0

210806-20:43:42,737 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.722379:
210806-20:43:42,738 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.722379:Determinant = 1.000000
210806-20:43:42,739 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.722379:Left-Right orientation: preserved
210806-20:43:42,739 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.722379:
210806-20:43:42,741 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.741804:Forward half transform =
210806-20:43:42,742 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.741804:0.999997 0.002179 0.000071 0.025019 
210806-20:43:42,743 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.741804:-0.002179 0.999990 -0.003773 -0.018855 
210806-20:43:42,743 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.741804:-0.000080 0.003773 0.999993 -0.232205 
210806-20:43:42,744 nipype.interface INFO:
	 stdout 2021-08-06T20:43:42.741804:0.000000 0.000000 0.000000 1.000000 
210806-20:43:42,745 nipype.interface INFO:
	 stdout 

210806-20:43:43,211 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.205038:
210806-20:43:43,409 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.409149:Rotation & Translation Matrix:
210806-20:43:43,412 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.409149:0.999993 0.003464 0.001088 0.066953 
210806-20:43:43,413 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.409149:-0.003457 0.999974 -0.006327 -0.103470 
210806-20:43:43,414 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.409149:-0.001110 0.006323 0.999979 -0.261498 
210806-20:43:43,414 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.409149:0.000000 0.000000 0.000000 1.000000 
210806-20:43:43,415 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.409149:
210806-20:43:43,416 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.409149:Rotation Angles (x,y,z) [rads] = -0.006327 -0.001088 0.003464 
210806-20:43:43,417 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.409149:
210806-20:43:43,418 nipype.interface IN

210806-20:43:43,891 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.877474:
210806-20:43:43,891 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.877474:Determinant = 1.000000
210806-20:43:43,892 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.877474:Left-Right orientation: preserved
210806-20:43:43,893 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.877474:
210806-20:43:43,894 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.877474:Forward half transform =
210806-20:43:43,895 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.877474:0.999999 0.001476 0.000564 0.050525 
210806-20:43:43,896 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.877474:-0.001474 0.999994 -0.003129 -0.073590 
210806-20:43:43,896 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.877474:-0.000568 0.003128 0.999995 -0.127493 
210806-20:43:43,897 nipype.interface INFO:
	 stdout 2021-08-06T20:43:43.877474:0.000000 0.000000 0.000000 1.000000 
210806-20:43:43,898 nipype.interface INFO:
	 stdout 

KeyboardInterrupt: 

In [ ]:
aryCurr = np.array(lstsubMot)
aryCurr_Ses =  aryCurr.reshape((aryCurr.size,-1))
aryCurr_TR = np.array(lstTR_sub)
aryCurr_TR_Ses = aryCurr_TR.reshape((aryCurr_TR.size,-1))
aryCurr_Nme = np.array(lstsubMot_Nme)
aryCurr_Nme_Ses = aryCurr_Nme.reshape((aryCurr_Nme.size,-1))
aryIdx = np.arange(1,len(aryCurr_Nme_Ses)+1)

aryCurr_mod = np.array(modalityList)
aryCurr_mod = aryCurr_mod.reshape((aryCurr_mod.size,-1))

aryCurr_subname = np.array(sub_list)
aryCurr_subs =  aryCurr_subname.reshape((aryCurr_subname.size,-1))

aryCurr_runsList = np.array(runsList)
aryCurr_runs =  aryCurr_runsList.reshape((aryCurr_runsList.size,-1))


     
data_dict = {
    'subject': aryCurr_subs[:,0],
    'Time/TR': aryCurr_TR_Ses[:,0],
    'Motion_Name': aryCurr_Nme_Ses[:,0],
    'Motion': aryCurr_Ses[:,0],
    'idx':aryIdx,
    'modality': aryCurr_mod[:,0],
    'run': aryCurr_runs[:,0]}
    
pd_ses = pd.DataFrame(data=data_dict)

In [ ]:
pd_ses.to_csv(root + '/derivatives/quality_assessmen/motionParametersStim.csv', index=False)  

### Plotting motion

For visual inspection, we can proceed with plotting the motion traces over time.

In [ ]:
subs_lim = ['sub-02']

for sub in subs_lim:
    pd_ses_rot_VASO = pd_ses.loc[(pd_ses['Motion_Name'].str.contains("R") == 1)&(pd_ses['subject']==sub)&(pd_ses['modality']=='VASO')].dropna()
    pd_ses_trans_VASO = pd_ses.loc[(pd_ses['Motion_Name'].str.contains("T") == 1)&(pd_ses['subject']==sub)&(pd_ses['modality']=='VASO')].dropna()
    
    pd_ses_rot_BOLD = pd_ses.loc[(pd_ses['Motion_Name'].str.contains("R") == 1)&(pd_ses['subject']==sub)&(pd_ses['modality']=='BOLD')].dropna()
    pd_ses_trans_BOLD = pd_ses.loc[(pd_ses['Motion_Name'].str.contains("T") == 1)&(pd_ses['subject']==sub)&(pd_ses['modality']=='BOLD')].dropna()
    

In [ ]:
fig, axes = plt.subplots(2, 1,sharex=True,figsize=(20,12))
plt.suptitle('%s Motion Summary [T:mm / R: Rads]'%(sub))


sns.lineplot(ax=axes[0], x='Time/TR',y='Motion',data=pd_ses_trans_VASO,hue='Motion_Name', palette = 'Set1')
sns.lineplot(ax=axes[0], x='Time/TR',y='Motion',data=pd_ses_trans_BOLD,hue='Motion_Name', palette = 'Set2')


axes[0].set_ylabel("translation (mm)")


sns.lineplot(ax=axes[1], x='Time/TR',y='Motion',data=pd_ses_rot_VASO,hue='Motion_Name', palette = 'Set1')
sns.lineplot(ax=axes[1], x='Time/TR',y='Motion',data=pd_ses_rot_BOLD,hue='Motion_Name', palette = 'Set2')



axes[1].set_ylabel("rotation (radians)")
plt.savefig('/home/sebastian/Desktop/%s_motion.jpg'%(sub))

### Z-transforming motion parameters

To make the motion traces usable for our GLM analysis, we have to z-transform them (why actually?).

In [ ]:
for sub in subs:
    TX_Z = stats.zscore(pd_ses['Motion'].loc[(pd_ses['Motion_Name']=='TX')&(pd_ses['subject']==sub)])
    TY_Z = stats.zscore(pd_ses['Motion'].loc[(pd_ses['Motion_Name']=='TY')&(pd_ses['subject']==sub)])
    TZ_Z = stats.zscore(pd_ses['Motion'].loc[(pd_ses['Motion_Name']=='TZ')&(pd_ses['subject']==sub)])

    RX_Z = stats.zscore(pd_ses['Motion'].loc[(pd_ses['Motion_Name']=='RX')&(pd_ses['subject']==sub)])
    RY_Z = stats.zscore(pd_ses['Motion'].loc[(pd_ses['Motion_Name']=='RY')&(pd_ses['subject']==sub)])
    RZ_Z = stats.zscore(pd_ses['Motion'].loc[(pd_ses['Motion_Name']=='RZ')&(pd_ses['subject']==sub)])
    
    TX_Z_new = []
    TY_Z_new = []
    TZ_Z_new = []
    
    RX_Z_new = []
    RY_Z_new = []
    RZ_Z_new = []

    for n in range(len(TX_Z)):
        TX_Z_new.append(TX_Z[n])
        TX_Z_new.append((TX_Z[n]+TX_Z[n])/2)
        
        TY_Z_new.append(TY_Z[n])
        TY_Z_new.append((TY_Z[n]+TY_Z[n])/2)
        
        TZ_Z_new.append(TZ_Z[n])
        TZ_Z_new.append((TZ_Z[n]+TZ_Z[n])/2)
        
        
        RX_Z_new.append(RX_Z[n])
        RX_Z_new.append((RX_Z[n]+RX_Z[n])/2)
        
        RY_Z_new.append(RY_Z[n])
        RY_Z_new.append((RY_Z[n]+RY_Z[n])/2)
        
        RZ_Z_new.append(RZ_Z[n])
        RZ_Z_new.append((RZ_Z[n]+RZ_Z[n])/2)
    
    sub_motion_matrix = pd.DataFrame({'TX_Z':TX_Z_new, 'TY_Z':TY_Z_new, 'TZ_Z':TZ_Z_new, 'RX_Z':RX_Z_new, 'RY_Z':RY_Z_new, 'RZ_Z':RZ_Z_new})
    sub_motion_matrix.to_csv( '/media/sebastian/elements/data/s1_anfunco_analysis/%s/func/vaso/rest_3b/motion/%s_motionparams.txt'%(sub,sub), header=None, index=None, sep=' ')

In [ ]:
# load saved file

sub_motion_matrix = pd.read_csv('/media/sebastian/elements/data/s1_anfunco_analysis/sub05/func/vaso/rest_3b/motion/sub05_motionparams.txt', header=None,  sep=' ') 
# Preview the first 5 lines of the loaded data 
sub_motion_matrix.columns = ['TX_Z', 'TY_Z', 'TZ_Z', 'RX_Z', 'RY_Z', 'RZ_Z']

sub_motion_matrix.head()

In [ ]:
TX_Z_new = []

for n in range(len(TX_Z)):
    TX_Z_new.append(TX_Z[n])
    TX_Z_new.append((TX_Z[n]+TX_Z[n])/2)

**Importantly**, we also have to upsample the motion traces because we calculated them on the "raw" data, while the preprocessed data is temporally upsampled. 

In [ ]:
f = interpolate.interp2d(TX_Z, kind='linear')


## Quality Assessment

Especially in high resolution fMRI, rigurous quality controls are necessary to evaulate the data. In the following we will go through various procedures to give a rough estimate of the data-quality.

- Mean image
- TSNR
- skew
- kurtosis
- FD
- Motion assessment a la Marquardt

For some of these measures, we have to do a bit more processing. Therefore, I will first continue with the steps outlined in Renzo's BOLD-correction scripts.

### Mean image, tSNR, skew and kurtosis

in this [blog-post](https://layerfmri.com/2020/04/06/qa/), Renzo describes different measures of quality assessment for layer fMRI.


**IMPORTANT**

This process uses a mask containing the postcentral gyrus plus a lot of other stuff. Therefore, it can be made more precise by using a GM-mask.

In [ ]:
measures = ['mean', 'tSNR','skew','kurt']


for sub in subs:
    for measure in 
    for modality in ['BOLD_intemp', 'VASO_LN']:
        data = nb.load('/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/sub-02/func/rest/sub-02_rest3b_run-001_VASO_LN_tSNR.nii').get_fdata()

In [ ]:
%%capture
measures = ['mean', 'tSNR']
colorMaps = [cm.Greys_r,'hot']



for modality in ['BOLD_intemp', 'VASO_LN']:
    for measure,colorMap in zip(measures, colorMaps):
        fig, axes = plt.subplots(len(subs), 1, figsize=(5,40), sharex=True, sharey=True)

        for sub,k in zip(subs,range(len(subs))):
            data = nb.load(f'/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/{sub}/func/rest/{sub}_rest3b_run-001_{modality}_{measure}.nii').get_fdata()
            data = data[:,:,10,0]
            rotated_img = ndimage.rotate(data, 90)
            im = axes[k].imshow(rotated_img, cmap=colorMap)
            plt.axis('off')
            #plt.colorbar()
            #fig.colorbar(rotated_img, ax=axes[k])
            axes[k].axis('off')
            if measure == 'tSNR':
                cbar = fig.colorbar(im,ax=axes[k], ticks=[rotated_img.min(), rotated_img.max()])
                cbar.ax.tick_params(size=0)
                
        plt.savefig(f'/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/qualityAssessment/{modality}_{measure}_map', bbox_inches = 'tight', pad_inches = 0)

In [ ]:
%%capture
measures = ['skew','kurt']
colorMaps = [cm.Greys_r, cm.Greys_r]



for modality in ['BOLD_intemp', 'VASO_LN']:
    for measure,colorMap in zip(measures, colorMaps):
        fig, axes = plt.subplots(len(subs), 1, figsize=(5,40), sharex=True, sharey=True)

        for sub,k in zip(subs,range(len(subs))):
            data = nb.load(f'/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/{sub}/func/rest/{sub}_rest3b_run-001_{modality}_{measure}.nii').get_fdata()
            data = data[:,:,10,0]
            rotated_img = ndimage.rotate(data, 90)
            #plt.colorbar()
            #fig.colorbar(rotated_img, ax=axes[k])
            axes[k].axis('off')

            im = axes[k].imshow(rotated_img, cmap=colorMap, vmin = -1, vmax = 1)
            

            cbar = fig.colorbar(im,ax=axes[k], ticks=[-1, 1])
            cbar.ax.tick_params(size=0)




                
        plt.savefig(f'/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/qualityAssessment/{modality}_{measure}_map', bbox_inches = 'tight', pad_inches = 0)

In [ ]:
subList = []
valList = []
modalityList = []
measureList = []


for sub in subs:
    for modality in ['BOLD_intemp', 'VASO_LN']:
        
        for measure in ['tSNR','skew','kurt']:

            data = nb.load(root + f'/derivatives/{sub}/func/rest/{sub}_rest3b_run-001_{modality}_{measure}.nii').get_fdata()


            mask = nb.load(root + f'/derivatives/{sub}/{sub}_moma.nii').get_fdata()

            data_masked = np.multiply(data[:,:,:,0], mask)[np.multiply(data[:,:,:,0], mask)!=0]


            subList.append([str(sub) for i in range(len(data_masked))])
            modalityList.append([str(modality) for i in range(len(data_masked))])
            measureList.append([str(measure) for i in range(len(data_masked))])
            valList.append(data_masked)

subList = [item for sublist in subList for item in sublist]
modalityList = [item for sublist in modalityList for item in sublist]
measureList = [item for sublist in measureList for item in sublist]
valList = [item for sublist in valList for item in sublist]


subList = np.array(subList).flatten()
subList = subList.reshape((subList.size,-1)) 

valList = np.array(valList).flatten()
valList = valList.reshape((valList.size,-1)) 

measureList = np.array(measureList).flatten()
measureList = measureList.reshape((measureList.size,-1)) 

modalityList = np.array(modalityList).flatten()
modalityList = modalityList.reshape((modalityList.size,-1)) 
    

QA = pd.DataFrame({'subject': subList[:,0], 'val': valList[:,0], 'measure': measureList[:,0], 'modality': modalityList[:,0]})

In [ ]:
def add_mean_line(data, var=None, **kws):
    
    # If no variable provided skip adding mean line
    if not var: return
    
    #Calculate mean for each group
    m = np.mean(data[var])
    
    #Get current axis
    ax = plt.gca()
    
    #add line at group mean
    ax.axvline(m, color='black', lw=2, ls='--')
    
    #annotate group mean
    #x_pos=0.65
    #if m > 5000: x_pos=0.2
    #ax.text(x_pos, 0.7, f'mean={m:.0f}', 
    #        transform=ax.transAxes,   #transforms positions to range from (0,0) to (1,1)
    #        color='maroon', fontweight='bold', fontsize=12)

In [ ]:
g = sns.FacetGrid(QA.loc[(QA['measure']=='tSNR')], row='subject',hue='modality')
g.map_dataframe(sns.kdeplot, "val", fill=True)
g.map_dataframe(add_mean_line, var='val')
g.add_legend()

In [ ]:
measures = ['tSNR','skew','kurt']


for measure,n in zip(measures, range(len(measures))):
    fig, axes = plt.subplots(len(subs), 1, figsize=(5,40), sharex=True, sharey=True)

    for sub,k in zip(subs,range(len(subs))):
            sns.kdeplot(ax = axes[k], data= QA.loc[(QA['measure']==measure)&(QA['subject']==sub)], x="val", hue='modality', fill=True)           
            #axes[0].set_title(f'{measure}', fontsize=24)
            axes[-1].set_xlabel(f'{measure}', fontsize=20)
            if measure != 'tSNR':
                axes[0].set_xlim(-1,1)
                
                
                
    plt.savefig(f'/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/qualityAssessment/{measure}', bbox_inches = 'tight', pad_inches = 0)

### Compute framewise displacements

In [ ]:
sub_FD = []
timepoints = []
subjects=[]
mods = []

for sub in subs:
    
    for modality in modalities:
        
        TX = pd_ses['Motion'].loc[(pd_ses['Motion_Name'].str.contains("TX") == 1)&(pd_ses['subject']==sub)&(pd_ses['modality']==modalitiesDict[modality])].tolist()
        TY = pd_ses['Motion'].loc[(pd_ses['Motion_Name'].str.contains("TY") == 1)&(pd_ses['subject']==sub)&(pd_ses['modality']==modalitiesDict[modality])].tolist()
        TZ = pd_ses['Motion'].loc[(pd_ses['Motion_Name'].str.contains("TZ") == 1)&(pd_ses['subject']==sub)&(pd_ses['modality']==modalitiesDict[modality])].tolist()

        RX = pd_ses['Motion'].loc[(pd_ses['Motion_Name'].str.contains("RX") == 1)&(pd_ses['subject']==sub)&(pd_ses['modality']==modalitiesDict[modality])].tolist()
        RY = pd_ses['Motion'].loc[(pd_ses['Motion_Name'].str.contains("RY") == 1)&(pd_ses['subject']==sub)&(pd_ses['modality']==modalitiesDict[modality])].tolist()
        RZ = pd_ses['Motion'].loc[(pd_ses['Motion_Name'].str.contains("RZ") == 1)&(pd_ses['subject']==sub)&(pd_ses['modality']==modalitiesDict[modality])].tolist()

        for n in range(len(TX)-1):
            FD_trial = abs(TX[n]-TX[n+1])+abs(TY[n]-TY[n+1])+abs(TZ[n]-TZ[n+1])+abs((50*RX[n])-(50*RX[n+1]))+abs((50*RY[n])-(50*RY[n+1]))+abs((50*RZ[n])-(50*RZ[n+1]))
            sub_FD.append(FD_trial)
            timepoints.append(n)
            subjects.append(sub)
            mods.append(modalitiesDict[modality])
        

FDs = pd.DataFrame({'subject':subjects, 'volume':timepoints, 'FD':sub_FD, 'modality': mods})

In [ ]:
sns.color_palette("tab10")[0]

In [ ]:
fig, axes = plt.subplots(len(subs), 1, figsize=(5,40), sharex=True, sharey=True)

for sub,k in zip(subs,range(len(subs))):
        sns.violinplot(ax = axes[k], data= FDs.loc[FDs['subject']==sub], inner = 'quartile', x='subject',y='FD', hue='modality', split=True, alpha = 0.5, palette = [sns.color_palette("tab10")[1], sns.color_palette("tab10")[0]])           
        axes[-1].set_xlabel(f'Framewise discplacement', fontsize=20)
        axes[k].set_ylabel(f'FD (mm)', fontsize=20)
        
        if not k == len(subs):
            axes[k].set_xticks([])
            axes[k].set_xlabel('')
        
        #if not k == len(subs):
         #   axes[k].legend([],[], frameon=False)
            





plt.savefig(f'/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/qualityAssessment/FDs', bbox_inches = 'tight', pad_inches = 0)

In [ ]:
fig  = plt.figure(figsize=(12,8))
plt.title('Framewise Displacements (rest)', fontsize=24)

sns.violinplot(x='subject',y='FD',data=FDs, hue = 'modality')
plt.ylabel("FD (mm)", fontsize=20)
plt.xlabel("participant", fontsize=20)

# plt.savefig('/home/sebastian/Desktop/resting_state_FD.jpg')

In [ ]:
fig, axes = plt.subplots(len(subs), 1,sharex=True,sharey=True, figsize = (20,10))

for sub,n in zip(subs,range(len(subs))):
    sns.lineplot(ax = axes[n], x='volume',y='FD',data=FDs.loc[FDs['subject']==sub],hue = 'modality')



# plt.savefig('/home/sebastian/Desktop/resting_state_FD_subplots.jpg')

In [ ]:
fig  = plt.figure(figsize=(12,8))
plt.title('Framewise Displacements')

sns.violinplot(x='subject',y='FD',data=FDs, inner="stick")
plt.ylabel("FD (mm)")

plt.savefig('/home/sebastian/Desktop/resting_state_FD.jpg')

## Anatomical processing


### Averaging of the 3 high resolution slabs using ITK-SNAP

To increase the SNR, I first averaged the 3 high resolution slabs we acquired.

In [3]:
root = '/media/sebastian/Data/S1ANFUNCO/raw_data'

In [10]:
restRuns = sorted(glob.glob(root + '/S1ANFUNCO_BIDS/sub-*/ses-00*/func/sub-*_ses-00*_task-rest*_cbv.nii.gz'))
subList = [i[-47:-41] for i in restRuns]

In [ ]:
for sub in subList[1:]:
    
    # Define output folder
    outFolder = f'/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/{sub}/anat'
    
    # look for 3 anatomical images.
    imgs = sorted(glob.glob(root + f'S1ANFUNCO_BIDS/{sub}/ses-00*/anat/*_ses-00*_highres-mp2rage_run-00*_uni.nii.gz'))
    
    # Read reference image.
    fixed = ants.image_read(imgs[0])
    
    # Save it for easy processing in a step below
    ants.image_write(fixed, outFolder + f'/1.nii', ri=False)
    
    # Register images 2 and 3
    for img,n in zip(imgs[1:],range(2,len(imgs)+1)):
        
        moving = ants.image_read(img)
        mytx = ants.registration(fixed=fixed , moving=moving, type_of_transform='Rigid' )
        warped_moving = mytx['warpedmovout']
        
        # Save it for easy processing in a step below
        ants.image_write(warped_moving, outFolder + f'/{n}.nii', ri=False)
    
    # Load 3 anatomical images for averaging
    img1 = nb.load(outFolder + f'/1.nii').get_fdata()
    img2 = nb.load(outFolder + f'/2.nii').get_fdata()
    img3 = nb.load(outFolder + f'/3.nii').get_fdata()

    # Average 3 anatomical images to increase SNR
    avg = np.add(img1, img2)
    avg = np.add(avg, img3)
    avg = np.divide(avg, 3)
    ni_img = nb.Nifti1Image(avg, nb.load(outFolder + f'/1.nii').affine)
    nb.save(ni_img, outFolder+ f'/{sub}_highres-mp2rage_uni_average.nii')
    
    # Bias field correction
    image = ants.image_read(outFolder+ f'/{sub}_highres-mp2rage_uni_average.nii')
    image_n4 = ants.n4_bias_field_correction(image)
    
    # Save corrected image
    ants.image_write(image_n4, outFolder+ f'/{sub}_highres-mp2rage_uni_average_N4corrected.nii', ri=False)
    
    # Delete intermediate steps.
    for n in range(1,len(imgs)+1):
        os.remove(outFolder + f"/{n}.nii")

In [ ]:
# For sub-02 the acquisition of anatomical data was a bit messy due to the various sessions, 
# therefore I will do it manually here.

outFolder = f'/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/sub-02/anat'

# Bias field correction
image = ants.image_read(outFolder+ f'/sub-02_highres-mp2rage_uni_average.nii.gz')
image_n4 = ants.n4_bias_field_correction(image)
    
# Save corrected image
ants.image_write(image_n4, outFolder+ f'/sub-02_highres-mp2rage_uni_average_N4corrected.nii', ri=False)

### Voxel reduction

Because we are only interested in the postcentral gyrus, we will reduce our voxels with several measures.
First, we will use brain masks and then crop our data.

#### Brain mask

Automatic brain extraction is often problematic. Therefore, I manually drew outlines of the brain in every 3rd slice and dilated them with the following code:

In [ ]:
for sub in subList:
    outFolder = f'/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/{sub}/anat'
    # load data
    nii = load(outFolder + f'/{sub}_highres-mp2rage_uni_average_N4corrected_brainMask.nii.gz')

    basename = nii.get_filename().split(os.extsep, 1)[0]
    dirname = os.path.dirname(nii.get_filename())
    data = nii.get_fdata()

    # perform closing
    data = morphology.binary_dilation(data, iterations=3)
    data = morphology.binary_erosion(data, iterations=3)

    # perform opening
    #data = morphology.binary_erosion(data, iterations=1)
    #data = morphology.binary_dilation(data, iterations=1)

    # save as nifti
    out = Nifti1Image(data, header=nii.header, affine=nii.affine)
    save(out, basename + "_opening_closing.nii.gz")


    print('Morphology operations are done.')


Then we can multiply the anatomical image with the mask.

In [ ]:
for sub in subList:
    outFolder = f'/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/{sub}/anat'
    anatData = nb.load(outFolder+ f'/{sub}_highres-mp2rage_uni_average_N4corrected.nii').get_fdata()
    maskData = nb.load(outFolder + f'/{sub}_highres-mp2rage_uni_average_N4corrected_brainMask_opening_closing.nii.gz').get_fdata()
    
    # Multiply with binary brain mask to get rid of non-brain voxels.
    masekdData = np.multiply(anatData, maskData)
    
    # save as nifti
    ni_img = nb.Nifti1Image(masekdData, nb.load(outFolder+ f'/{sub}_highres-mp2rage_uni_average_N4corrected.nii').affine)
    nb.save(ni_img, outFolder+ f'/{sub}_highres-mp2rage_uni_average_N4corrected_brain.nii')


#### Cropping

To further reduce our voxels, we copped the data to our area of interest. This involved some visual inspection. The coordinates.

In [ ]:
boundariesDict = {'sub-02': [80, 170, 75, 115, 5, 51],
                  'sub-05': [110, 160, 60, 125, 7, 47],
                  'sub-06': [100, 160, 70, 90, 9, 46],
                  'sub-07': [100, 180, 70, 120, 10, 45],
                  'sub-09': [75, 175, 110, 90, 7, 47],
                  'sub-10': [90, 175, 80, 100, 9, 45],
                  'sub-12': [90, 175, 60, 110, 4, 48],
                  'sub-15': [85, 165, 80, 110, 4, 49],
                  'sub-16': [90, 160, 65, 125, 4, 51],
                  'sub-17': [80, 155, 95, 115, 4, 48],
                  'sub-18': [65, 170, 90, 115, 4, 51]
                 }



In [ ]:
from nipype.interfaces.fsl import ExtractROI

for sub in subList:
    outFolder = f'/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/{sub}/anat'

    anatFile = outFolder+ f'/{sub}_highres-mp2rage_uni_average_N4corrected_brain.nii'

    # Apparently, the fslroi wrapper in nipype wants an existing file as output filename (?). Therefore, I will create one with the same dimensions.
    tmpFile = nb.load(anatFile)
    tmp = Nifti1Image(tmpFile.get_fdata(), header=tmpFile.header, affine=tmpFile.affine)
    save(tmp, outFolder+ f'/{sub}_highres-mp2rage_uni_average_N4corrected_brain_trunc.nii.gz')


    fslroi = ExtractROI(in_file=anatFile, roi_file= outFolder+ f'/{sub}_highres-mp2rage_uni_average_N4corrected_brain_trunc.nii.gz', 
                        x_min = boundariesDict[sub][0], x_size = boundariesDict[sub][1], 
                        y_min = boundariesDict[sub][2], y_size = boundariesDict[sub][3],
                        z_min = boundariesDict[sub][4], z_size = boundariesDict[sub][5]
                       )

    out = fslroi.run()

### Segmentation

We started the segmentation process with an automatic segmentation using FSL FAST:

In [ ]:
from nipype.interfaces import fsl
for sub in sublistLim:
    fastr = fsl.FAST()
    fastr.inputs.in_files = outFolder+ f'/{sub}_highres-mp2rage_uni_average_N4corrected_brain_trunc.nii.gz'
    out = fastr.run()

The segmetntation was then manually corrected and saved as

'{sub}_highres-mp2rage_uni_average_N4corrected_brain_trunc_pveseg_corrected.nii.gz'

In [ ]:
from nipype.interfaces import afni

for sub in subList:
    inputFiles = sorted(glob.glob(f'/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/{sub}/func/*/{sub}_*_run-00*_moco_*ulled.nii'))
    tcat = afni.TCat()
    tcat.inputs.in_files = inputFiles
    tcat.inputs.out_file= f'/media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/{sub}/func/combined.nii'

    res = tcat.run()

In [ ]:
%%bash -s "{" ".join(subList)}"

List1=$1

subsList=($List1)

len=${#subsList[@]}
 

for (( j=0; j<$len; j++ ))
do

echo Current subject: ${subsList[$j]}

cd /media/sebastian/Data/S1ANFUNCO/raw_data/derivatives/${subsList[$j]}/func

3dTstat -cvarinv -overwrite -prefix ${subsList[$j]}_EPI_T1w.nii combined.nii
rm combined.nii

N4BiasFieldCorrection -d 3 -i ${subsList[$j]}_EPI_T1w.nii -o ${subsList[$j]}_EPI_T1w_N4Corrected.nii


echo "Upsampling"
delta_x=$(3dinfo -di ${subsList[$j]}_EPI_T1w_N4Corrected.nii)
delta_y=$(3dinfo -dj ${subsList[$j]}_EPI_T1w_N4Corrected.nii)
delta_z=$(3dinfo -dk ${subsList[$j]}_EPI_T1w_N4Corrected.nii)
sdelta_x=$(echo "((sqrt($delta_x * $delta_x) / 5))"|bc -l)
sdelta_y=$(echo "((sqrt($delta_y * $delta_y) / 5))"|bc -l)
sdelta_z=$(echo "((sqrt($delta_z * $delta_z) / 1))"|bc -l)
# here I only upscale in 2 dimensions.

3dresample -dxyz $sdelta_x $sdelta_y $sdelta_z -rmode Cu -overwrite -prefix ${subsList[$j]}_EPI_T1w_N4Corrected_scaled.nii -input ${subsList[$j]}_EPI_T1w_N4Corrected.nii

done

### Registration

For the registration, I manually matched the average UNI images with the upsampled anatomy in EPI space. The resulting matrix was saved as "initial_matrix".

To further improve our registration, we will use a registration mask. Fortunately, we happen to have one already from the initial motion correction. We just have to upsample it so it machtes out target space.

We'll do that like so:

In [ ]:
%%bash -s "{" ".join(subs)}" "$root"

for sub in $1
do
cd $2/derivatives/$sub

echo Upsampling $sub moma
delta_x=$(3dinfo -di "$sub"_moma.nii)
delta_y=$(3dinfo -dj "$sub"_moma.nii)
delta_z=$(3dinfo -dk "$sub"_moma.nii)
sdelta_x=$(echo "((sqrt($delta_x * $delta_x) / 5))"|bc -l)
sdelta_y=$(echo "((sqrt($delta_y * $delta_y) / 5))"|bc -l)
sdelta_z=$(echo "((sqrt($delta_z * $delta_z) / 1))"|bc -l)
# here I only upscale in 2 dimensions.

3dresample -dxyz $sdelta_x $sdelta_y $sdelta_z -rmode Cu -overwrite -prefix "$sub"_moma_scaled.nii -input "$sub"_moma.nii

done

In [12]:
%%bash -s "{" ".join(subList)}" "$root"

List1=$1

subsList=($List1)

len=${#subsList[@]}

for (( j=0; j<$len; j++ ))
do

cd $2/derivatives/${subsList[$j]}/anat/registration

echo Current participant: ${subsList[$j]}
echo Fixed: $2/derivatives/${subsList[$j]}/func/${subsList[$j]}_EPI_T1w_N4Corrected_scaled.nii
echo Moving: ../${subsList[$j]}_highres-mp2rage_uni_average_N4corrected_brain_trunc.nii.gz
echo ###

antsRegistration \
--verbose 1 \
--dimensionality 3  \
--float 0  \
--collapse-output-transforms 1  \
--interpolation BSpline[5] \
--output [registered1_,registered1_Warped.nii,1]  \
--use-histogram-matching 0  \
--winsorize-image-intensities [0.005,0.995]  \
--initial-moving-transform initial_matrix.txt \
--transform SyN[0.1,3,0]  \
--metric CC[$2/derivatives/${subsList[$j]}/func/${subsList[$j]}_EPI_T1w_N4Corrected_scaled.nii, ../${subsList[$j]}_highres-mp2rage_uni_average_N4corrected_brain_trunc.nii.gz,1,2]  \
--convergence [60x10,1e-6,10]  \
--shrink-factors 2x1  \
--smoothing-sigmas 1x0vox  \
-x ../../${subsList[$j]}_moma_scaled.nii

antsApplyTransforms \
--interpolation BSpline[5] \
-d 3 -i ../${subsList[$j]}_highres-mp2rage_uni_average_N4corrected_brain_trunc.nii.gz \
-r $2/derivatives/${subsList[$j]}/func/${subsList[$j]}_EPI_T1w_N4Corrected_scaled.nii \
-t registered1_1Warp.nii.gz \
-t registered1_0GenericAffine.mat \
-o ${subsList[$j]}_highres-mp2rage_uni_average_N4corrected_brain_trunc_registered.nii

done

Process is interrupted.


In [ ]:
%%bash -s "{" ".join(subs)}" "{" ".join(outFolders)}"


List1=$1
List2=$2

subsList=($List1)
outFolders=($List2)


len=${#subsList[@]}


imageTypes=("n" "nn")




for (( j=0; j<$len; j++ ))
do

# base=${subsList[$j]}_rest3b
base=${subsList[$j]}_rest3b_run-001
outFolder=${outFolders[$j]}
echo Current subject: ${subsList[$j]}
echo The output will be saved here: ${outFolder}
echo The basename of the output is: ${base}
echo #
echo #
echo #